In [55]:
"""
Load search data
"""
import cPickle
results = cPickle.load(open('tmp/results.pickle', 'r'))

In [56]:
"""
Extract web corpus

文節ユニグラム。以下で取得
http://s-yata.jp/corpus/nwc2010/ngrams/
"""
import math
corpus = {}
with open('tmp/1gm-0000', 'r') as f:
    for line in f.readlines():
        word, number = line.split('\t')
        word = word.decode('utf8')
        # Take a log of the frequency to prepare for tf-idf
        number = math.log(int(number))
        corpus[word] = number

In [72]:
"""
Pre-processing functions
"""
import MeCab as mecab

HIRAGANA = [unichr(i) for i in xrange(12353,12353+86)]
HIRAGANA_STR = ''.join(HIRAGANA)

EXCLUDE_LIST = [
    "、",
    "◯",
    "✕",
    "「",
    "」",
    "(",
    ")",
    "（",
    "）",
    ".",
    ",",
    "-",
    ":",
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
    "!",
    "?",
    "%",
    "*",
    "+",
    "=",
    "~",
    "-",
    "_",
    "&",
    "[",
    "]"
]

def is_all_hiragana(s):
    for c in s:
        if c not in HIRAGANA_STR:
            return False
    return True

def is_to_exclude(s):
    for e in EXCLUDE_LIST:
        if e in s:
            return True
    s = s.decode('utf8')
    if is_all_hiragana(s):
        return True
    return False

def parser(text):
    """形態素パーサ
    名詞と形容詞を返す。
    また、is_to_excludeで指定された近視語を考慮する。
    """
    tagger = mecab.Tagger()
    text = unicode(text)
    encoded_text = text.encode('utf-8')

    node = tagger.parseToNode(encoded_text)
    while node:
        pos = node.feature.split(",")[0]
        if pos in ["名詞", "形容詞"] and not is_to_exclude(node.surface):
            yield node.surface.decode('utf-8')
        node = node.next

        
def get_tfidf(article):
    """Function for calculating tf-idf with DF taken from the Web corpus
    """
    features = {}
    for word in parser(article):
        df = corpus.get(word, None)
        if df:
            tf = article.count(word)
            features[word] = 1.0 * tf / df
        else:
            continue
    return sorted(features.items(), key=lambda x: x[1], reverse=True)

In [74]:
"""
Compare to the original to create suggestions
"""

original = u"""
60秒で元気になれる耳寄りヘルスケア
2014.7.8
「すごくつらい訳ではないけど、毎日どことなくだるい」「足腰が弱くなった」「朝がつらい」

20代後半から忍び寄る「疲れ」という名の万病の元、皆さんも少なからず経験があるのではないでしょうか？　「病気」とは診断されず、また健康診断でも再検査を受けるほどではない、でもつらい疲れの解消には、肝臓と腎臓の回復が効果的です。 

倦怠感や頭が重い症状の解消（肝臓回復）

早起きが苦手な人によく見られるのが、肝臓が弱っているというケースです。肝臓は身体の解毒をしているのですが、疲れがたまっているということは、この解毒がうまくいかず老廃物が溜まっている可能性があります。肝臓の機能を回復させるには、以下の食材が効果を発揮します。

・イカ、貝類、魚

栄養ドリンクで必ずといっていいほどうたわれている「タウリン」は、肝臓の細胞の回復を促進します。イカ、貝類、魚には、このタウリンが自然の状態で豊富に含まれています。

・ささみ、魚、大豆

肝臓を修理する材料となる、良質なアミノ酸とタンパク質をとることができます。

・青魚、くるみ、えごま油

肝臓の負担になっているのが、コレステロール。青魚、くるみ、えごま油には、コレステロールを排出させる「オメガ3脂肪酸」が豊富に含まれています。このオメガ3脂肪酸は限られた食材にしか豊富には含まれず、常に不足しがちな栄養成分ですので、意識して摂取してみてください。
倦怠感やむくみの調整（腎臓回復）

病気になってもはっきりとした自覚症状が現れにくいため「沈黙の臓器」といわれる腎臓ですが、やはり疲れているとだるさやむくみが発生しやすくなるので、早めのケアが肝腎です。
※ここで挙げるのは腎臓回復の食事なので、腎臓病の方には当てはまりません！

・果物、野菜

果物や野菜に含まれるカリウムが、身体の水分調整や老廃物の排出を手伝います。これからの季節では、スイカなどがカリウムを多く含む食材の代表格です。

・くるみ、ゴマ

中国医学で「腎精」（腎臓を元気にする）と称されるほど、腎臓の健康に役立つと言われています。腎臓は血液のろ過をする器官ですが、くるみやゴマは血液の流れを良くし、老廃物の排出を促すことで腎臓の回復力を高めます。

・山芋、自然薯、ごぼう、レンコン

根菜や芋類にはカリウムと食物繊維が多く含まれており、この2つの成分が動物性の老廃物を排出する手助けをしてくれます。ただし、腎臓病などでカリウムの摂取制限が必要な場合は、根菜や芋類は避けてください。
"""


top_terms = set()

# Look for the defining terms in top 20 ranked pages
for result in results:
    for page in result.results[:20]:
        top = get_tfidf(page.body)[:5]
        for item in top:
            word = item[0]
            if word not in top_terms and word not in original:
                top_terms.add(word)

print(u', '.join(top_terms))

秋口, 土曜, 接骨, やる気, 月, サプリメント, 不全, 夏, 療法, ホルモン, Consuelo, Streptozotocin, in, 減, 目, 原, タイプ, 疲労, 眠, functions, 厚, フィルター, mayami, ダル, スマホ, 脳, /, 治療, 夏バテ, 労, 茶, インゲン, 妊娠, 心当たり, 尿, 意外, 私, 睡眠, 胃, 時, Huffington, 運動, マメ, 筋, RA, 紹介, 不眠症, 院, 神経, 日常, 種類, 時間, 酷使, レンズ, 血圧, taiken, on, 低下, 更年期, 障害, harima, Cooked, 塩, 異常, 阻害, 食欲, エネルギー, p, 残暑, 何気ない, 法, 熟眠, 袋原, がまん強く, 原因


In [75]:
"""
検索ワード毎に、その中での特徴量を取ってみる
（同じ検索ワードで表示されるページはもともと似ているから、tf-idfの母集団をWeb全体ではなく検索結果に限定してみる）
"""
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer=parser)

first = results[0]
texts = [page.body for page in first.results]

x = vectorizer.fit_transform(texts)
terms = vectorizer.get_feature_names()
tfidfs = x.toarray()
print(tfidfs)

[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.19771226 ...,  0.          0.          0.        ]
 [ 0.          0.          0.19771226 ...,  0.          0.          0.        ]
 [ 0.          0.          0.19771226 ...,  0.          0.          0.        ]]


In [76]:
"""
各記事のtfidfトップ３の単語を使い、オリジナルの記事にない単語をピックアップ
"""
top_terms = set()
for page_tfidf in tfidfs:
    # get indices
    indices = [i[0] for i in sorted(enumerate(page_tfidf), key=lambda x:x[1], reverse=True)]
    # get top 3
    top = [terms[i] for i in indices[:3]]
    for term in top:
        if term not in top_terms and term not in original:
            top_terms.add(term)

print(u', '.join(top_terms))

寝汗, 症候群, リウマチ, 硬く, カウンセラー, 整体, 花粉, 休日, 黄体, タイプ, 疲労, 受信, 禁物, 丸, 妊娠, <, 私, 土曜, >, 低下, 奈美, 脊髄, net, 衰弱, shy, 痙攣, 更新, 感受性, 異常, 不定, 食道, 療法, 生理, 不眠, 柔軟, 悩み, 化学, 鼻腔, 筋肉, /, 自律, ;, 髄, ビタミン, 電波, イビキ, 汗, アレルギー, 沈着, うつ病, 冬眠, PMS, 卵, 熱中, 安静, ウンチ, 市, cancer, 期間, 眠く, サバイバー, 閉経, 疾患, 受付, ", 活動, 治療, 横, system, 患者, 肩こり, 悪い, 自分, 糖尿, 夕方, 研究, 食後, 夏, 種類, 心臓, 盛り, cancertherapy, ensp, 全身, 矯正, 分解, 眠気, 赤ちゃん, 袋原, 慢性, 調査, vol, 消化, Invalid, 炎, ホルモン, 甲状腺, 本多, 頭痛, 温度, 施術, ドキドキ, 月曜日, 冬, 夏バテ, 五月, 鼻, キーワード, 睡眠, ママ, 相手, 大腸, 器質, 逆流, 神経, 時間, 情動, ">, ID, 修復, 更年期, 心不全, 便秘, cutting, 愁訴, 整骨, 自立, パソコン, CFS
